[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/GAN-for-generating-MNIST-images/blob/master/Generative_adversial_network_to_generate_MNIST_images.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
os.chdir('drive/Generative adversial network to generate mnist images')


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


DC_D_W1 = tf.Variable(xavier_init([5, 5, 1, 16]))
DC_D_b1 = tf.Variable(tf.zeros(shape=[16]))

DC_D_W2 = tf.Variable(xavier_init([3, 3, 16, 32]))
DC_D_b2 = tf.Variable(tf.zeros(shape=[32]))

DC_D_W3 = tf.Variable(xavier_init([7 * 7 * 32, 128]))
DC_D_b3 = tf.Variable(tf.zeros(shape=[128]))

DC_D_W4 = tf.Variable(xavier_init([128, 1]))
DC_D_b4 = tf.Variable(tf.zeros(shape=[1]))

theta_DC_D = [DC_D_W1, DC_D_b1, DC_D_W2, DC_D_b2, DC_D_W3, DC_D_b3, DC_D_W4, DC_D_b4]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def dc_discriminator(x):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, DC_D_W1, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b1)
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, DC_D_W2, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b2)
    conv2 = tf.reshape(conv2, shape=[-1, 7 * 7 * 32])
    h = tf.nn.relu(tf.matmul(conv2, DC_D_W3) + DC_D_b3)
    logit = tf.matmul(h, DC_D_W4) + DC_D_b4
    prob = tf.nn.sigmoid(logit)

    return prob, logit


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


G_sample = generator(Z)
D_real, D_logit_real = dc_discriminator(X)
D_fake, D_logit_fake = dc_discriminator(G_sample)

# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_logit_real, labels = tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_logit_fake, labels = tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_logit_fake, labels =tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_DC_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('./data/MNIST_data', one_hot=True)

sess = tf.Session()
sess.run(tf.initialize_all_variables())


i = 0

for it in range(1000000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter: 0
D loss: 7.687
G_loss: 39.46

Iter: 1000
D loss: 0.001491
G_loss: 10.35

Iter: 2000
D loss: 0.04146
G_loss: 26.33

Iter: 3000
D loss: 0.1536
G_loss: 11.3

Iter: 4000
D loss: 0.0577
G_loss: 13.

Iter: 28000
D loss: 0.007652
G_loss: 5.922

Iter: 29000
D loss: 0.02505
G_loss: 6.92

Iter: 30000
D loss: 0.005588
G_loss: 7.318

Iter: 31000
D loss: 0.02787
G_loss: 5.608

Iter: 32000
D loss: 0.01936
G_loss: 6.597

Iter: 33000
D loss: 0.01057
G_loss: 7.046

Iter: 34000
D loss: 0.006311
G_loss: 6.651

Iter: 35000
D loss: 0.01591
G_loss: 7.171

Iter: 36000
D loss: 0.006737
G_loss: 8.29

Iter: 37000
D loss: 0.001518
G_loss: 7.749

Iter: 38000
D loss: 0.001067
G_loss: 7.302

Iter: 39000
D loss: 0.0001877
G_loss: 9.377

Iter: 40000
D loss: 0.002369
G_loss: 9.934

Iter: 41000
D loss: 0.0006453
G_loss: 9.254

Iter: 42000
D loss: 0.0001039
G_loss: 11.85

Iter: 43000
D loss: 0.0005159
G_loss: 12.66

Iter: 44000
D loss: 0.0001865
G_loss: 10.89

Iter: 45000
D loss: 0.0003641
G_loss: 10.82

Iter: 46000
D loss: 2.894e-06
G_loss: 24.05

Iter: 47000
D loss: 6.241e-05
G_loss: 18.33

Iter: 48000
D loss: 0.0145
G_loss: 10.93

Iter: 49000
D loss: 0.0001566
G_loss: 18.67

Iter: 50000
D loss: 9.025e-07
G_

Iter: 94000
D loss: 0.06363
G_loss: 5.084

Iter: 95000
D loss: 0.04826
G_loss: 5.592

Iter: 96000
D loss: 0.05294
G_loss: 5.43

Iter: 97000
D loss: 0.09112
G_loss: 5.061

Iter: 98000
D loss: 0.06954
G_loss: 5.106

Iter: 99000
D loss: 0.05168
G_loss: 5.202

Iter: 100000
D loss: 0.02167
G_loss: 5.119

Iter: 101000
D loss: 0.01718
G_loss: 6.315

Iter: 102000
D loss: 0.063
G_loss: 6.396

Iter: 103000
D loss: 0.04624
G_loss: 5.713

Iter: 104000
D loss: 0.1287
G_loss: 6.252

Iter: 105000
D loss: 0.01555
G_loss: 6.037

Iter: 106000
D loss: 0.01264
G_loss: 6.33

Iter: 107000
D loss: 0.0146
G_loss: 5.915

Iter: 108000
D loss: 0.03808
G_loss: 7.172

Iter: 109000
D loss: 0.00629
G_loss: 6.202

Iter: 110000
D loss: 0.1823
G_loss: 5.852

